In [1]:
import pandas as pd
import numpy as np
import math
import os
from tqdm.notebook import tqdm

In [2]:
df = pd.read_csv('test.csv', index_col=0)
concealed_index = np.load('test_conceal_index.npy')

In [24]:
from utils import entry_to_desc
from openai import OpenAI
client = OpenAI(api_key=os.environ['OPENAI_KEY'])

def df_to_desc(df_):
    descs = []
    for i in tqdm(df_.index):
        entry = df_.loc[i]
        desc = entry_to_desc(entry)
        descs.append((i, desc))
    return descs

def df_concealed_to_desc(df_):
    descs = []
    for row, i in enumerate(df_.index):
        entry = df_.loc[i]
        if concealed_index[row] == 0:
            entry['Total Price'] = float('nan')
        elif concealed_index[row] == 1:
            entry['Unit Price'] = float('nan')
        elif concealed_index[row] == 2:
            entry['Creation Date'] = float('nan')
        elif concealed_index[row] == 3:
            entry['Purchase Date'] = float('nan')
        elif concealed_index[row] == 4:
            entry['Quantity'] = float('nan')
        else:
            raise Exception
        desc = entry_to_desc(entry)
        descs.append((i, desc))
    return descs

In [26]:
embedding_model = "text-embedding-ada-002"
embedding_length = 1536

def df_to_embeddings(df_, output):
    if os.path.isfile(output):
        return []
    n_rows, _ = df_.shape
    embeddings = np.empty((n_rows, embedding_length+1))
    for row, i in enumerate(df_.index):
        entry = df_.loc[i]
        if concealed_index[row] == 0:
            entry['Total Price'] = float('nan')
        elif concealed_index[row] == 1:
            entry['Unit Price'] = float('nan')
        elif concealed_index[row] == 2:
            entry['Creation Date'] = float('nan')
        elif concealed_index[row] == 3:
            entry['Purchase Date'] = float('nan')
        elif concealed_index[row] == 4:
            entry['Quantity'] = float('nan')
        else:
            raise Exception
        desc = entry_to_desc(entry)
        response = client.embeddings.create(input=desc, model=embedding_model)
        embeddings[row, 0] = i
        embeddings[row, 1:] = np.array(response.data[0].embedding)
    np.save(output, embeddings)
    print(f'Processed {output}')
    return embeddings

In [28]:
import multiprocess as mp

with mp.Pool(20) as pool:
    # Initialize tqdm with the total number of tasks
    total_tasks = 100
    per_task = 10
    args_list = [(df[j*per_task:(j+1)*per_task], f'test_embeddings/test_embeddings_{j}.npy') for j in range(total_tasks)]
    result = pool.starmap_async(df_to_embeddings, args_list)
    # Wait for all processes to finish
    emb = result.get()

/var/folders/7w/gkxkn5vs7t3_lv3qlp7c094h0000gn/T/ipykernel_89148/3642344541.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entry['Purchase Date'] = float('nan')
/var/folders/7w/gkxkn5vs7t3_lv3qlp7c094h0000gn/T/ipykernel_89148/3642344541.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entry['Purchase Date'] = float('nan')
/var/folders/7w/gkxkn5vs7t3_lv3qlp7c094h0000gn/T/ipykernel_89148/3642344541.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning

Processed test_embeddings/test_embeddings_24.npy
Processed test_embeddings/test_embeddings_10.npy
Processed test_embeddings/test_embeddings_32.npy
Processed test_embeddings/test_embeddings_2.npy
Processed test_embeddings/test_embeddings_4.npy
Processed test_embeddings/test_embeddings_0.npy
Processed test_embeddings/test_embeddings_26.npy
Processed test_embeddings/test_embeddings_22.npy
Processed test_embeddings/test_embeddings_20.npy
Processed test_embeddings/test_embeddings_30.npy
Processed test_embeddings/test_embeddings_8.npy
Processed test_embeddings/test_embeddings_36.npy
Processed test_embeddings/test_embeddings_6.npy
Processed test_embeddings/test_embeddings_14.npy
Processed test_embeddings/test_embeddings_34.npy
Processed test_embeddings/test_embeddings_28.npy
Processed test_embeddings/test_embeddings_38.npy
Processed test_embeddings/test_embeddings_12.npy
Processed test_embeddings/test_embeddings_16.npy
Processed test_embeddings/test_embeddings_18.npy
Processed test_embeddings

/var/folders/7w/gkxkn5vs7t3_lv3qlp7c094h0000gn/T/ipykernel_89148/3642344541.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entry['Purchase Date'] = float('nan')


Processed test_embeddings/test_embeddings_11.npy


/var/folders/7w/gkxkn5vs7t3_lv3qlp7c094h0000gn/T/ipykernel_89148/3642344541.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entry['Purchase Date'] = float('nan')


...

Processed test_embeddings/test_embeddings_63.npy
Processed test_embeddings/test_embeddings_73.npy
Processed test_embeddings/test_embeddings_43.npy


/var/folders/7w/gkxkn5vs7t3_lv3qlp7c094h0000gn/T/ipykernel_89148/3642344541.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entry['Unit Price'] = float('nan')
/var/folders/7w/gkxkn5vs7t3_lv3qlp7c094h0000gn/T/ipykernel_89148/3642344541.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entry['Unit Price'] = float('nan')
/var/folders/7w/gkxkn5vs7t3_lv3qlp7c094h0000gn/T/ipykernel_89148/3642344541.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vie

Processed test_embeddings/test_embeddings_71.npy
Processed test_embeddings/test_embeddings_75.npy


/var/folders/7w/gkxkn5vs7t3_lv3qlp7c094h0000gn/T/ipykernel_89148/3642344541.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entry['Unit Price'] = float('nan')
/var/folders/7w/gkxkn5vs7t3_lv3qlp7c094h0000gn/T/ipykernel_89148/3642344541.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entry['Unit Price'] = float('nan')
/var/folders/7w/gkxkn5vs7t3_lv3qlp7c094h0000gn/T/ipykernel_89148/3642344541.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vie

Processed test_embeddings/test_embeddings_82.npy
Processed test_embeddings/test_embeddings_78.npy
Processed test_embeddings/test_embeddings_86.npy
Processed test_embeddings/test_embeddings_84.npy
Processed test_embeddings/test_embeddings_88.npy
Processed test_embeddings/test_embeddings_57.npy
Processed test_embeddings/test_embeddings_80.npy
Processed test_embeddings/test_embeddings_96.npy
Processed test_embeddings/test_embeddings_98.npy
Processed test_embeddings/test_embeddings_92.npy
Processed test_embeddings/test_embeddings_94.npy
Processed test_embeddings/test_embeddings_76.npy
Processed test_embeddings/test_embeddings_90.npy
Processed test_embeddings/test_embeddings_79.npy
Processed test_embeddings/test_embeddings_83.npy
Processed test_embeddings/test_embeddings_87.npy
Processed test_embeddings/test_embeddings_81.npy
Processed test_embeddings/test_embeddings_89.npy
Processed test_embeddings/test_embeddings_85.npy
Processed test_embeddings/test_embeddings_97.npy
Processed test_embed